# EDA on Airbnb Listings in Toronto
The research interest lines in what makes a good Airbnb listing in Toronto

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [3]:
listings = pd.read_csv("listings.csv")
row, col = listings.shape
row
col

74

In [4]:
# Split training and testing dataset
train_index_end = int(row*0.75)
train_indx = np.arange(train_index_end)
test_indx = np.arange(train_index_end+1, row)
train_index_end, test_indx[-1]

(11313, 15083)

In [5]:
train_df = listings.iloc[train_indx]
test_df = listings.iloc[test_indx]

In [6]:
train_df.shape

(11313, 74)

In [7]:
train_df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [8]:
# Drop variables that are highly likely irrelevant or difficult to process
var_to_drop = [
       'listing_url', 'scrape_id', 'last_scraped', 'description',
       'neighborhood_overview', 'picture_url', 'neighbourhood', 'latitude',
       'longitude', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped'      
]

train_df.drop(var_to_drop, inplace=True, axis = 1)
train_df.shape

C:\Users\wjx\AppData\Local\Temp\ipykernel_7028\2812972004.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(var_to_drop, inplace=True, axis = 1)


(11313, 50)

## Get Data Type and Integrity Information

In [9]:
train_df.iloc[:,0:10].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11313 entries, 0 to 11312
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  11313 non-null  int64 
 1   name                11312 non-null  object
 2   host_id             11313 non-null  int64 
 3   host_url            11313 non-null  object
 4   host_name           11307 non-null  object
 5   host_since          11307 non-null  object
 6   host_location       11303 non-null  object
 7   host_about          6587 non-null   object
 8   host_response_time  5588 non-null   object
 9   host_response_rate  5588 non-null   object
dtypes: int64(2), object(8)
memory usage: 972.2+ KB


* Name has one missing value
* Neighbor overview has nearly 4000 missing values
* IDs are integer valued

In [10]:
train_df.iloc[:,10:20].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11313 entries, 0 to 11312
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   host_acceptance_rate       5776 non-null   object 
 1   host_is_superhost          11307 non-null  object 
 2   host_thumbnail_url         11307 non-null  object 
 3   host_picture_url           11307 non-null  object 
 4   host_neighbourhood         9706 non-null   object 
 5   host_listings_count        11307 non-null  float64
 6   host_total_listings_count  11307 non-null  float64
 7   host_verifications         11313 non-null  object 
 8   host_has_profile_pic       11307 non-null  object 
 9   host_identity_verified     11307 non-null  object 
dtypes: float64(2), object(8)
memory usage: 972.2+ KB


* All variables in this range have null values
* host acceptance rate, neighborhood are the variables with the most missing values. Possibly they are not required to create a profile. 
* The other with no or just a few missing values.
* Host verification is non-null, maybe every host need to be verified on ID
* host_has_profile_pic should be boolean, is boolean object?
* Many variables are url, for simplicity, **url will be removed**.

In [11]:
train_df.iloc[:,20:30].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11313 entries, 0 to 11312
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   neighbourhood_cleansed        11313 non-null  object 
 1   neighbourhood_group_cleansed  0 non-null      float64
 2   property_type                 11313 non-null  object 
 3   room_type                     11313 non-null  object 
 4   accommodates                  11313 non-null  int64  
 5   bathrooms                     0 non-null      float64
 6   bathrooms_text                11306 non-null  object 
 7   bedrooms                      10509 non-null  float64
 8   beds                          11271 non-null  float64
 9   amenities                     11313 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 972.2+ KB


* None of entries of neighbourhood group cleansed and bathrooms columns are populated with values. Remove them
* The others have only a few null values, fill NA is not big problem

In [12]:
train_df.iloc[:,30:40].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11313 entries, 0 to 11312
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   price                      11313 non-null  object 
 1   number_of_reviews          11313 non-null  int64  
 2   number_of_reviews_ltm      11313 non-null  int64  
 3   number_of_reviews_l30d     11313 non-null  int64  
 4   first_review               9501 non-null   object 
 5   last_review                9501 non-null   object 
 6   review_scores_rating       9501 non-null   float64
 7   review_scores_accuracy     9330 non-null   float64
 8   review_scores_cleanliness  9332 non-null   float64
 9   review_scores_checkin      9329 non-null   float64
dtypes: float64(4), int64(3), object(3)
memory usage: 972.2+ KB


* From price to number of review in the last 30 days are non-null variables
* The remaining have around 1500 missing values, they need completion 
* Price should be float or integer, but it is object, may be $ is included

In [13]:
train_df.iloc[:,40:50].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11313 entries, 0 to 11312
Data columns (total 10 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   review_scores_communication                   9329 non-null   float64
 1   review_scores_location                        9328 non-null   float64
 2   review_scores_value                           9329 non-null   float64
 3   license                                       2633 non-null   object 
 4   instant_bookable                              11313 non-null  object 
 5   calculated_host_listings_count                11313 non-null  int64  
 6   calculated_host_listings_count_entire_homes   11313 non-null  int64  
 7   calculated_host_listings_count_private_rooms  11313 non-null  int64  
 8   calculated_host_listings_count_shared_rooms   11313 non-null  int64  
 9   reviews_per_month                             9501 non-null  

* License hastoo many missing values, should remove license
* Scores have around 1800 missing values
* Different types of listing count have no null values

### Decisions
* Remove columns: neighbouhood group cleansed, bathrooms, license
* Keep variables with non-null values as low as 5500, correlate them with good host. And use the correlation to impose the missing values
* Remove variables with _url in their names
* Convert multiple review scores into more synthesized score.
* Host about might be a long paragraph of text which is difficult to process, we will leave it out

#### Correcting by Removing Bad Data

In [14]:
null_col = ["neighbourhood_group_cleansed", "bathrooms", "license"]
url_col = list(filter(lambda x: "_url" in x, train_df.columns))
col_to_remove = null_col + url_col + ["host_about"]
col_to_remove

['neighbourhood_group_cleansed',
 'bathrooms',
 'license',
 'host_url',
 'host_thumbnail_url',
 'host_picture_url',
 'host_about']

In [15]:
train_df.drop(col_to_remove, axis=1, inplace=True)

C:\Users\wjx\AppData\Local\Temp\ipykernel_7028\1318187045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(col_to_remove, axis=1, inplace=True)


In [16]:
row, col = train_df.shape
row, col

(11313, 43)

#### Convert Price to Numerical Data

In [17]:
train_df.price.head(10).values

array(['$469.00', '$94.00', '$72.00', '$45.00', '$75.00', '$125.00',
       '$100.00', '$70.00', '$70.00', '$130.00'], dtype=object)

In [18]:
# Delete $ and convert to float64
numerical_price = train_df.price.str.extract(r'(\d+)\.(\d+)')
numerical_price = numerical_price.loc[:, 0]
numerical_price

0        469
1         94
2         72
3         45
4         75
        ... 
11308     89
11309     40
11310     99
11311    109
11312     55
Name: 0, Length: 11313, dtype: object

In [19]:
train_df.price = numerical_price
train_df.price

C:\Users\wjx\AppData\Local\Temp\ipykernel_7028\955045270.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.price = numerical_price


0        469
1         94
2         72
3         45
4         75
        ... 
11308     89
11309     40
11310     99
11311    109
11312     55
Name: price, Length: 11313, dtype: object

In [20]:
# Set type to integer
train_df.price = train_df.price.astype('int64')

C:\Users\wjx\AppData\Local\Temp\ipykernel_7028\2929202183.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.price = train_df.price.astype('int64')


#### Combining Scores into One Unified Metrics

* Rating is the overall experience of travelers, assign 0.025
* Cleaness is a big part, assign 0.2
* Accuracy relates to faithful representation, assign 0.2
* Communication relects attitude and warmth,  assign 0.15
* Check-in shouldn't be a big problem in most places, assign 0.025
* Value means cost-effectiveness, assign 0.2
* Location affect safety, noise, and convenience, assign 0.2

In [23]:
combine_score = (train_df.review_scores_rating*0.025 + 
train_df.review_scores_accuracy*0.2 + 
train_df.review_scores_cleanliness*0.2 + 
train_df.review_scores_checkin*0.05 + 
train_df.review_scores_communication*0.15+ 
train_df.review_scores_location*0.175+ 
train_df.review_scores_value*0.2)

In [26]:
combine_score[:10], combine_score.info()

<class 'pandas.core.series.Series'>
Int64Index: 11313 entries, 0 to 11312
Series name: None
Non-Null Count  Dtype  
--------------  -----  
9328 non-null   float64
dtypes: float64(1)
memory usage: 176.8 KB


(0    5.00000
 1    4.86650
 2    4.71875
 3    4.89450
 4    4.90550
 5        NaN
 6    4.70125
 7    4.07500
 8    4.78775
 9        NaN
 dtype: float64,
 None)

In [37]:
train_df[list(filter(lambda x: "review_scores" in x, train_df.columns.values))].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11313 entries, 0 to 11312
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   review_scores_rating         9501 non-null   float64
 1   review_scores_accuracy       9330 non-null   float64
 2   review_scores_cleanliness    9332 non-null   float64
 3   review_scores_checkin        9329 non-null   float64
 4   review_scores_communication  9329 non-null   float64
 5   review_scores_location       9328 non-null   float64
 6   review_scores_value          9329 non-null   float64
dtypes: float64(7)
memory usage: 707.1 KB


Seems that at least 9328 cells are populated with data for review scores, and exactly 9328 units are completely populated with data for the combined score.

In [43]:
score_band = pd.Series(map(lambda x: "good" if x >= 4 else ("fair" if x >= 3 else "poor"), combine_score))
score_band.head(6)

0    good
1    good
2    good
3    good
4    good
5    poor
dtype: object

In [44]:
train_df["score_band"] = score_band
train_df.drop(list(filter(lambda x: "review_scores" in x, train_df.columns.values)), axis = 1, inplace=True)
train_df.columns

C:\Users\wjx\AppData\Local\Temp\ipykernel_7028\2989994410.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["score_band"] = score_band
C:\Users\wjx\AppData\Local\Temp\ipykernel_7028\2989994410.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(list(filter(lambda x: "review_scores" in x, train_df.columns.values)), axis = 1, inplace=True)


Index(['id', 'name', 'host_id', 'host_name', 'host_since', 'host_location',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'first_review', 'last_review', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month',
       'score_band'],
      dtype='object')

## Describe the Data

In [50]:
train_df.iloc[:,0:10].describe(include="O")

,name,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost
count,11312,11307,11307,11303,5588,5588,5776,11307
unique,11171,4032,2855,293,4,65,87,2
top,Private room in a shared hostel suite downtown,David,2011-07-06,"Toronto, Ontario, Canada",within an hour,100%,100%,f
freq,8,89,49,8769,2802,3716,1751,8249


* There are duplicate names of listings, meaning that the same listing is rented twice or more
* Host name shows duplicates. The guy named "David" had 89 listing records
* Host_since should be date object, need to check if the type is desired
* The locations are more concentrated, 11303 listing to 293 unique locations. The top value seems not indicating accurate street address, need to further inpect value behaviour
* Host response time seems to have only a few unique values, and it shows in text like "within an hour." Need to further inspect it
* Response rate and acceptance rate are shown as text due % sign. Need to inspect values and remove % sign.
* Need to convert host_is_superhost into boolean values

### Correcting object-valued variables in the first 10 variables

In [62]:
train_df.host_name.unique

<bound method Series.unique of 0             Alexandra
1        Kathie & Larry
2         Yohan & Sarah
3                 Brent
4                  Rita
              ...      
11308            Bozica
11309             Cathy
11310            Bozica
11311            Elaine
11312            Irmita
Name: host_name, Length: 11313, dtype: object>

In [74]:
test_date = train_df.host_since.values[0]
type(test_date)

str

In [79]:
train_df.host_since = pd.to_datetime(train_df.host_since, infer_datetime_format=True)

C:\Users\wjx\AppData\Local\Temp\ipykernel_7028\43717234.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.host_since = pd.to_datetime(train_df.host_since, infer_datetime_format=True)


In [81]:
train_df.host_response_time.unique()

array([nan, 'within a few hours', 'within an hour', 'within a day',
       'a few days or more'], dtype=object)

In [85]:
#Since host response time is labeled in an ordinal order, we can convert it to ordinal values
train_df.host_response_time = train_df.host_response_time.map(
    {'within a few hours':1, 'within an hour':2, 'within a day':3,
    'a few days or more':4, np.NaN: 4})
train_df.host_response_time

C:\Users\wjx\AppData\Local\Temp\ipykernel_7028\1512940098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.host_response_time = train_df.host_response_time.map(


0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
11308   NaN
11309   NaN
11310   NaN
11311   NaN
11312   NaN
Name: host_response_time, Length: 11313, dtype: float64

In [86]:
train_df.host_acceptance_rate.unique()

array([nan, '54%', '100%', '82%', '50%', '97%', '0%', '96%', '94%', '92%',
       '75%', '33%', '99%', '80%', '61%', '86%', '95%', '68%', '91%',
       '18%', '65%', '90%', '98%', '72%', '71%', '69%', '67%', '93%',
       '40%', '27%', '73%', '76%', '57%', '10%', '85%', '64%', '56%',
       '83%', '81%', '47%', '74%', '62%', '17%', '70%', '88%', '22%',
       '25%', '79%', '89%', '30%', '63%', '42%', '41%', '20%', '78%',
       '60%', '29%', '35%', '77%', '52%', '14%', '87%', '55%', '46%',
       '38%', '59%', '51%', '31%', '8%', '84%', '13%', '36%', '44%',
       '43%', '5%', '58%', '9%', '15%', '24%', '53%', '21%', '11%', '48%',
       '66%', '45%', '34%', '39%', '16%'], dtype=object)

In [94]:
train_df.host_acceptance_rate = train_df.host_acceptance_rate.str.rstrip("%")
train_df.host_acceptance_rate.unique()

C:\Users\wjx\AppData\Local\Temp\ipykernel_7028\1638827011.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.host_acceptance_rate = train_df.host_acceptance_rate.str.rstrip("%")


array([nan, '54', '100', '82', '50', '97', '0', '96', '94', '92', '75',
       '33', '99', '80', '61', '86', '95', '68', '91', '18', '65', '90',
       '98', '72', '71', '69', '67', '93', '40', '27', '73', '76', '57',
       '10', '85', '64', '56', '83', '81', '47', '74', '62', '17', '70',
       '88', '22', '25', '79', '89', '30', '63', '42', '41', '20', '78',
       '60', '29', '35', '77', '52', '14', '87', '55', '46', '38', '59',
       '51', '31', '8', '84', '13', '36', '44', '43', '5', '58', '9',
       '15', '24', '53', '21', '11', '48', '66', '45', '34', '39', '16'],
      dtype=object)

In [108]:
def rate_band(rate):
    if rate == np.NaN:
        return 4

    if rate >= 80:
        return 1
    elif rate >= 60:
        return 2
    else :
        return 3
indx_for_convertion = train_df.host_acceptance_rate.isnull()
train_df.host_acceptance_rate[indx_for_convertion] = train_df.host_acceptance_rate.loc[indx_for_convertion].astype("int32")

ValueError: cannot convert float NaN to integer

False

0         True
1         True
2         True
3         True
4        False
         ...  
11308     True
11309     True
11310     True
11311     True
11312    False
Name: host_acceptance_rate, Length: 11313, dtype: bool